# SEN163A – Assignment 2 - Large-scale Internet Data Analysis
**DEADLINE**: Friday 18 Febary 2022 before 18:00

## Group 14
- Antonio Sanchez Martin - ''5245834''
- Felix Unger - ''5673631''
- Jeroen van Paassen - ''4720970''
- Yunus Emre Torlak - '' 5597668''

## Remarks from the lab
- The BGP routing might be very important! Lecturer hinted at this.
- Need to learn about JSON objects.
- Use the wayback machine for getting older info.
- Look at `pickle` in python for storing the data.
  - Not human-readable
  - For the `.pkl` files.
- Question 1: 
  - I.e. is the data enough?
  - You don't have to answer the questions sequentially
- Use the library `time`
  - Remember to repeat measurements to get an accurate time (as every time the code is run, it can give different results)

## Questions
1. Evaluate if there are *limitations* in the provided **datasets** (AS and probe data set). If you find limitations, describe these and conjecture possible reasons, supported with data.
   - No need to do it at the beginning! 
2. With the *AS and probe data set*, **find the number *m* of AS’s that can be used for hosting** in the EU and have probes in the *RIPE data set*. Sort the **ASN’s in ascending order** and include the **first and last three** in your report (number, name and country).
3. For a **single hour** in the *RIPE data set*: find all valid entries where the probe has hosting *type AS* and the *target IPv4 is from an EU country*. Implement this in an efficient way.
4. Move from using only an hour to the **full day**. It is advisable to store the raw results of each file. Then, using all processed files, *calculate the average latency’s for each country-AS* combination and store the results into one $$n_{countries}\times m$$ matrix. If we could place one server in each country, what would the *minimum average latency be for each country*? (include in your report)
5. Since we are only allowed to place four servers, determine the **best four datacenters** based on the total latency for all countries. Report your findings and your procedure to obtain them. Also include the *average latency for each country*.

## Dataset description and loading

### Probe dataset
- It only has two columns:
  - `prb_id` -> Used to check if the probe is also in the RIPE dataset.
  - `ASN` -> Autonomous System Number (i.e. small-network id)
- Why? -> To join the RIPE and AS datasets


In [53]:
import pickle

with open("./datasets/probe_dataset.pkl", 'rb') as file:
    probe_df = pickle.load(file)

print(probe_df.shape)
probe_df.head()

(11008, 2)


,prb_id,ASN
0,1,AS3265
1,2,AS1136
2,3,AS3265
3,6,AS6830
4,8,AS3265


### AS Dataset
- 5 columns:
  - `ASN`
  - Country code
  - Network name
  - Total number of IPs in network
  - Type of network
- Why? -> Can give us the number of IPs and location

In [54]:
import pandas
AS_df = pandas.read_pickle("./datasets/AS_dataset.pkl")
AS_df = AS_df.rename(columns={"type": "type_AS"})
AS_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60122 entries, 0 to 60121
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ASN      60122 non-null  object
 1   Country  60122 non-null  object
 2   Name     60122 non-null  object
 3   NumIPs   60122 non-null  object
 4   type_AS  60122 non-null  object
dtypes: object(5)
memory usage: 2.3+ MB


### RIPE

In [55]:
import timeit
import pandas

file_location = './datasets/ping-2022-03-01T1000.bz2'
# Importing
ping_df = pandas.read_json(file_location, lines = True, nrows = 1e5)
ping_df = ping_df.rename(columns={"type": "type_ping"})
ping_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 26 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   fw         100000 non-null  int64         
 1   mver       95463 non-null   object        
 2   lts        100000 non-null  int64         
 3   dst_name   100000 non-null  object        
 4   af         100000 non-null  int64         
 5   dst_addr   99532 non-null   object        
 6   src_addr   99530 non-null   object        
 7   proto      100000 non-null  object        
 8   ttl        82723 non-null   float64       
 9   size       100000 non-null  int64         
 10  result     100000 non-null  object        
 11  dup        100000 non-null  int64         
 12  rcvd       100000 non-null  int64         
 13  sent       100000 non-null  int64         
 14  min        100000 non-null  float64       
 15  max        100000 non-null  float64       
 16  avg        100000 non

In [56]:
# Check how many "from" addresses are unique
ping_df['from'].nunique()

7161

In [57]:
%%script false --no-raise-error
# Time estimation -> Rather unreliable actually, it takes much less than that
## Note: The more lines imported in "one go", the more efficient it becomes
num_lines_to_test = 1e5
num_lines_file = sum(1 for _ in open(file_location, encoding = "ISO-8859-1")) # get the number of lines in the file
stmt = f'pandas.read_json("{file_location}", lines = True, nrows = {num_lines_to_test})' # what to run on timeit
setup = 'import pandas'
loading_time = timeit.timeit(stmt=stmt, setup=setup, number=100) # Measure how long it takes to load 100 lines 100 times on average
time_estimation = num_lines_file/num_lines_to_test*loading_time
print(f"The entire file would take {time_estimation/60} mins ({time_estimation/3600} hrs) to load")
print(f"The file has {num_lines_file} lines")


#### Things to check
- Are `dst_name` and `dst_addr` always the same? -> No!

In [58]:
# dst_name vs dst_addr
ping_df['dst_name'].equals(ping_df['dst_addr'])


False

The RIPE dataset contains ping measurements. This dataset contains a lot of information about these measurements that are executed by a select number of probes to most IPv4 addresses in the world. One row in this dataset (which can be seen in the table above) tells us about 

### IPv4 dataset
The IP addresses are in integer format.

Regarding the data structure of the IPv4-dataset, we are having four columns:
- `ip_from` describes the the lower boundary for a sample of IP-adresses
- `ip_to` describes the upper boundary for a sample of IP-adresses 
- `country_code` and `country_name` describe the country which belongs to the IP samples defined with `ip_from` and `ip_to`

Since the IP data was given as integeres, we converted them into IP adresses format.

In [59]:
import pandas as pd
ipv4_df = pd.read_csv("datasets/IP2LOCATION-LITE-DB1.CSV", names=["ip_from", "ip_to", "country_code", "country_name"])
ipv4_df.drop(index=ipv4_df.index[0], axis=0, inplace=True) # Drop the first line as it is not data
ipv4_df.head()

,ip_from,ip_to,country_code,country_name
1,16777216,16777471,US,United States of America
2,16777472,16778239,CN,China
3,16778240,16779263,AU,Australia
4,16779264,16781311,CN,China
5,16781312,16785407,JP,Japan


In [60]:
# Convert int ip to, well, IP format
import ipaddress
for ip in ('ip_to', 'ip_from'):
    ipv4_df[ip] = ipv4_df[ip].apply(ipaddress.ip_address)
ipv4_df.head()

,ip_from,ip_to,country_code,country_name
1,1.0.0.0,1.0.0.255,US,United States of America
2,1.0.1.0,1.0.3.255,CN,China
3,1.0.4.0,1.0.7.255,AU,Australia
4,1.0.8.0,1.0.15.255,CN,China
5,1.0.16.0,1.0.31.255,JP,Japan


## Question 2: How many addresses are within the EU on the probe, AS and RIPE datasets?
2. With the *AS and probe data set*, **find the number *m* of AS’s that can be used for hosting** in the EU and have probes in the *RIPE data set*. Sort the **ASN’s in ascending order** and include the **first and last three** in your report (number, name and country).

- Common features in each dataset:
  - AS number:
    - AS
    - Probe
  - Probe ID:
    - Probe
    - RIPE
  - Location:
    - AS

In [90]:
# Joining the Probe and AS datasets based on the ASN
combined_df = AS_df.merge(probe_df, how='inner', on='ASN')
combined_df.head()

,ASN,Country,Name,NumIPs,type_AS,prb_id
0,AS21246,AL,IPKO Telecommunications LLC,"77,824",business,1237
1,AS21246,AL,IPKO Telecommunications LLC,"77,824",business,21795
2,AS42313,AL,Albtelecom Sh.a.,"70,912",business,2851
3,AS42313,AL,Albtelecom Sh.a.,"70,912",business,11779
4,AS42313,AL,Albtelecom Sh.a.,"70,912",business,21678


In [113]:
# Join the RIPE with the combined DF based on prb_id
country_codes_df = pandas.read_csv("./datasets/eu_country_codes.csv") # EU countries with codes

final_df = combined_df.merge(ping_df, how='inner', on='prb_id') # Join based on prb_id
final_df = final_df[final_df["Country"].isin(country_codes_df["ISO2"])] # Only EU countries
final_df = final_df[final_df["Country"].isin(country_codes_df["ISO2"])] # Only EU countries
final_df.sort_values(by="ASN", ascending=True, inplace=True)
final_df.head()


,ASN,Country,Name,NumIPs,type_AS,prb_id,fw,mver,lts,dst_name,...,max,avg,msm_id,timestamp,msm_name,from,type_ping,step,group_id,ttr
31566,AS1101,NL,SURFnet bv,"1,838,592",isp,6031,5040,2.4.1,56,176.74.17.75,...,16.827102,16.452186,17575882,2022-03-01 10:02:28,Ping,145.220.0.55,ping,240.0,17575881.0,NaN
31448,AS1101,NL,SURFnet bv,"1,838,592",isp,6031,5040,2.4.1,27,2a01:5041:4efd::3,...,-1.000000,-1.000000,18412074,2022-03-01 10:01:59,Ping,2001:67c:6ec:201:145:220:0:55,ping,240.0,18412073.0,NaN
31447,AS1101,NL,SURFnet bv,"1,838,592",isp,6031,5040,2.4.1,26,78.189.206.22,...,56.335730,55.911498,31958502,2022-03-01 10:01:58,Ping,145.220.0.55,ping,240.0,31958501.0,NaN
31446,AS1101,NL,SURFnet bv,"1,838,592",isp,6031,5040,2.4.1,26,141.170.19.12,...,15.696840,15.558945,3577502,2022-03-01 10:01:58,Ping,145.220.0.55,ping,240.0,3577501.0,NaN
31445,AS1101,NL,SURFnet bv,"1,838,592",isp,6031,5040,2.4.1,26,178.250.2.47,...,1.051550,0.964205,31323747,2022-03-01 10:01:58,Ping,145.220.0.55,ping,240.0,31323746.0,NaN


In [124]:
# Top 3 (unique) elements
final_df.loc[:, ["ASN", "Name", "Country"]].drop_duplicates(subset=['ASN']).head(3)

,ASN,Name,Country
31566,AS1101,SURFnet bv,NL
31009,AS1103,SURFnet bv,NL
24839,AS1213,HEAnet,IE


In [123]:
# Bottom 3 (unique) elements
final_df.loc[:, ["ASN", "Name", "Country"]].drop_duplicates(subset=['ASN']).tail(3)

,ASN,Name,Country
48706,AS9161,Poundbury Systems Ltd.,GB
20530,AS9188,InfoServe GmbH,DE
20080,AS9211,Nawork Internet Informationssysteme GmbH,DE


In [122]:
# A quick test that this worked (i.e. that the merge actually behaves as expected)
prb_id = 6031
final_df[final_df['prb_id'] == prb_id].loc[:, ["prb_id", "ASN", "Name", "NumIPs", "Country", "type_AS"]]

,prb_id,ASN,Name,NumIPs,Country,type_AS
31566,6031,AS1101,SURFnet bv,"1,838,592",NL,isp
31448,6031,AS1101,SURFnet bv,"1,838,592",NL,isp
31447,6031,AS1101,SURFnet bv,"1,838,592",NL,isp
31446,6031,AS1101,SURFnet bv,"1,838,592",NL,isp
31445,6031,AS1101,SURFnet bv,"1,838,592",NL,isp
...,...,...,...,...,...,...
31510,6031,AS1101,SURFnet bv,"1,838,592",NL,isp
31511,6031,AS1101,SURFnet bv,"1,838,592",NL,isp
31512,6031,AS1101,SURFnet bv,"1,838,592",NL,isp
31513,6031,AS1101,SURFnet bv,"1,838,592",NL,isp


In [116]:
probe_df[probe_df['prb_id'] == prb_id].head()

,prb_id,ASN
1454,6031,AS1101


In [117]:
combined_df[combined_df['prb_id'] == prb_id].head()

,ASN,Country,Name,NumIPs,type_AS,prb_id
5937,AS1101,NL,SURFnet bv,"1,838,592",isp,6031


In [118]:
# 